In [1]:
import mysql.connector

In [41]:
cnx = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="*",
    database = "library")
cnx

In [21]:
cursor = cnx.cursor()
cursor

In [23]:
# find the number of availalbe copies of the book (Dracula)
# check total copies of the book
cursor.execute('''
               select Title, count(*) as availalbe_copies_of_Dracula from books
               where Title = 'Dracula';
               ''')
cursor.fetchall()

[('Dracula', 3)]

In [25]:
# current total loans of the book 
cursor.execute('''
                  select b.Title, count(*) Loaned_copies_of_Dracula from books as b
                  inner join loans as a
                  on b.BookID = a.BookID
                  where b.Title = 'Dracula';
               ''')
cursor.fetchall()


[('Dracula', 32)]

In [26]:
# total available books of dracula
cursor.execute('''
                select sum(copies_of_Dracula) as total_copies_of_Dracula from
                (select count(*) as copies_of_Dracula from books
                where Title = 'Dracula'
                union 
                select count(*) as copies_of_Dracula from books as b
                inner join loans as a
                on b.BookID = a.BookID
                where b.Title = 'Dracula') as new_table;
               ''')
cursor.fetchall()

[(Decimal('35'),)]

In [31]:
# /*******************************************************/
# /* Add new books to the library                        */
# /*******************************************************/
cursor.execute('''
INSERT INTO Books (BookID,Title,Author,Published,Barcode)
VALUES (202,'Fabless: The Transformation of the Semiconductor Industry','Daniel Nenni',2014,1497525047);
               ''')

In [32]:
cursor.execute('''
              select * from books
              order by BookID desc;
               ''')
cursor.fetchall()

[(202,
  'Fabless: The Transformation of the Semiconductor Industry',
  'Daniel Nenni',
  2014,
  1497525047),
 (201,
  'Semiconductor Physics And Devices: Basic Principles',
  'Donald A. Neamen',
  2011,
  73529583),
 (200, "A Doll's House", 'Henrik Ibsen', 1879, 2719805876),
 (199, 'Anna Karenina', 'graf Leo Tolstoy', 1878, 6318181860),
 (198, "A Doll's House", 'Henrik Ibsen', 1879, 6227932647),
 (197, 'Hard Times', 'Charles Dickens', 1854, 6616502217),
 (196,
  'The Awakening, and Selected Short Stories',
  'Kate Chopin',
  1899,
  5859811018),
 (195, 'Metamorphosis', 'Franz Kafka', 1915, 8949930384),
 (194, 'The Tale of Peter Rabbit', 'Beatrix Potter', 1901, 5705857485),
 (193, 'Candide', 'Voltaire', 1759, 9718482110),
 (192, 'The Turn of the Screw', 'Henry James', 1898, 9586004168),
 (191, 'Emma', 'Jane Austen', 1815, 5913123586),
 (190, 'Adventures of Huckleberry Finn', 'Mark Twain', 1884, 3766010638),
 (189, 'The Jungle Book', 'Rudyard Kipling', 1894, 2183505735),
 (188,
  'Trac

In [33]:
# /*******************************************************/
# /* Check out Books: books(4043822646, 2855934983) whose patron_email(jvaan@wisdompets.com), loandate=2020-08-25, duedate=2020-09-08, loanid=by_your_choice*/
# /*
cursor.execute('''
                  select * from books as b
                  inner join loans as l on b.BookID = l.BookID
                  inner join patrons as p on p.PatronID = l.PatronID
                  where b.Barcode in (4043822646, 2855934983)
                  and
                  p.Email = "jvaan@wisdompets.com"
                  and
                  l.LoanDate = "2020-08-25"
                  and
                  l.DueDate = "2020-09-08";
                ''')
cursor.fetchall()

[]

In [34]:
# /********************************************************/
# /* Check books for Due back                             */
# /* generate a report of books due back on July 13, 2020 */
# /* with patron contact information                      */
# /********************************************************/
cursor.execute('''
                  select * from books as b
                  inner join loans as l on b.BookID = l.BookID
                  inner join patrons as p on p.PatronID = l.PatronID
                  where l.DueDate = '2020-07-13';
                ''')
cursor.fetchall()


[(10,
  'The Count of Monte Cristo',
  'Alexandre Dumas',
  1884,
  1799238621,
  1996,
  10,
  99,
  '2020-06-29',
  '2020-07-13',
  None,
  'Kala',
  'Simonian',
  'ksimonian2q@landonhotel.com',
  99),
 (155,
  'The Prophet',
  'Kahlil Gibran',
  1923,
  9013040583,
  1997,
  155,
  47,
  '2020-06-29',
  '2020-07-13',
  None,
  'Lydie',
  'Awmack',
  'lawmack1a@samoca.org',
  47),
 (169,
  'Second Treatise of Government',
  'John Locke',
  1689,
  5647388501,
  1998,
  169,
  18,
  '2020-06-29',
  '2020-07-13',
  None,
  'Anthe',
  'Dinjes',
  'adinjesh@samoca.org',
  18),
 (73,
  'Dracula',
  'Bram Stoker',
  1897,
  8730298424,
  1999,
  73,
  59,
  '2020-06-29',
  '2020-07-13',
  None,
  'Cornelia',
  'Koppe',
  'ckoppe1m@landonhotel.com',
  59)]

In [35]:
# /*******************************************************/
# /* Return books to the library (which have barcode=6435968624) and return this book at this date(2020-07-05)                    */
# /*******************************************************/
cursor.execute('''
                  select * from books as b
                  inner join loans as l on b.BookID = l.BookID 
                  where b.Barcode = 6435968624
                  and
                  l.ReturnedDate = '2020-07-05';
                ''')
cursor.fetchall()


[]

In [36]:
# /*******************************************************/
# /* Encourage Patrons to check out books                */
# /* generate a report of showing 10 patrons who have
# checked out the fewest books.                          */
# /*******************************************************/

cursor.execute('''
                  select p.FirstName, p.LastName, p.Email, p.PatronID, count(*) as total_loaned_books from books as b
                  inner join loans as l on b.BookID = l.BookID
                  inner join patrons as p on p.PatronID = l.PatronID
                  group by p.PatronID
                  order by total_loaned_books, p.PatronID
                  limit 10;
               ''')
cursor.fetchall()

[('Alaster', 'Ruggles', 'aruggles1w@red30design.com', 69, 9),
 ('Caril', 'Matejic', 'cmatejic2a@orangevalleycaa.org', 83, 9),
 ('Merle', 'Sukbhans', 'msukbhans20@orangevalleycaa.org', 73, 11),
 ('Norby', 'Sleight', 'nsleight1r@kinetecoinc.com', 64, 12),
 ('Curr', 'Fittall', 'cfittall2l@orangevalleycaa.org', 94, 12),
 ('Jack', 'Vaan', 'jvaan@wisdompets.com', 50, 13),
 ('Aleen', 'Fasey', 'afasey2@kinetecoinc.com', 3, 14),
 ('Cammi', 'Kynett', 'ckynett6@orangevalleycaa.org', 7, 14),
 ('Orton', 'Stavers', 'ostaversb@orangevalleycaa.org', 12, 14),
 ('Carla', 'Swinfen', 'cswinfenf@kinetecoinc.com', 16, 14)]

In [37]:
# /*******************************************************/
# /* Find books to feature for an event                  
#  create a list of books from 1890s that are
#  currently available                                    */
# /*******************************************************/
cursor.execute('''
                  select * from books
                  where Published = 1890;
               ''')
cursor.fetchall()


[(7, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 9615072124),
 (11, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 2855934983),
 (55, 'An Occurrence at Owl Creek Bridge', 'Ambrose Bierce', 1890, 6432246701),
 (56, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 5057915174),
 (98, 'An Occurrence at Owl Creek Bridge', 'Ambrose Bierce', 1890, 6854696214),
 (128, 'The Picture of Dorian Gray', 'Oscar Wilde', 1890, 3664785630),
 (134, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 9535295098),
 (138, 'The Sign of the Four', 'Arthur Conan Doyle', 1890, 6652171593),
 (154,
  'An Occurrence at Owl Creek Bridge',
  'Ambrose Bierce',
  1890,
  8301235528)]

In [38]:
# /*******************************************************/
# /* Book Statistics 
# /* create a report to show how many books were 
# published each year.                                    */
# /*******************************************************/
cursor.execute('''
                  select Published, count(distinct(Title)) as total_books_published from books
                  group by Published
                  order by total_books_published desc;
               ''')
cursor.fetchall()


[(1890, 3),
 (1895, 3),
 (1847, 2),
 (1850, 2),
 (1859, 2),
 (1869, 2),
 (1884, 2),
 (1892, 2),
 (1898, 2),
 (1899, 2),
 (1911, 2),
 (1922, 2),
 (1811, 1),
 (1813, 1),
 (1815, 1),
 (1818, 1),
 (1839, 1),
 (1759, 1),
 (1849, 1),
 (1851, 1),
 (1852, 1),
 (1854, 1),
 (1858, 1),
 (1861, 1),
 (1865, 1),
 (1876, 1),
 (1878, 1),
 (1879, 1),
 (1880, 1),
 (1883, 1),
 (1532, 1),
 (1886, 1),
 (1887, 1),
 (1889, 1),
 (1605, 1),
 (1670, 1),
 (1894, 1),
 (1689, 1),
 (1896, 1),
 (1897, 1),
 (1726, 1),
 (1729, 1),
 (1900, 1),
 (1901, 1),
 (1902, 1),
 (1905, 1),
 (1906, 1),
 (1908, 1),
 (1375, 1),
 (1913, 1),
 (1914, 1),
 (1915, 1),
 (1920, 1),
 (1921, 1),
 (1791, 1),
 (1923, 1),
 (2011, 1),
 (2014, 1)]

In [39]:
# /*************************************************************/
# /* Book Statistics                                           */
# /* create a report to show 5 most popular Books to check out */
# /*************************************************************/
cursor.execute('''
                  select b.Title, count(*) as borrowed_copies_per_book from books as b
                  inner join loans as l on b.BookID = l.BookID
                  group by b.Title
                  order by borrowed_copies_per_book desc
                  limit 5;
               ''')
cursor.fetchall()

[('Frankenstein; Or, The Modern Prometheus', 53),
 ('The Awakening, and Selected Short Stories', 43),
 ('The War of the Worlds', 43),
 ('The Scarlet Letter', 42),
 ('The Picture of Dorian Gray', 39)]